# DP Aggregate Seeded Synthesizer
DP Aggregate Seeded synthesizer is a differentially private synthesizer that relies on DP Marginals to build synthetic data. It will compute DP Marginals (called aggregates) for your dataset using an specified `reporting length`, and synthesize data based on the computed aggregated counts.

> The `reporting length` is the maximum combination length that aggregates are going to be computed for.

## 1. Overview

### 1.1. Aggregated data generation with DP
Let's consider the following input as example:

| A | B | C |
| -- | -- | -- |
| a1 | b1 | c1 |
| a1 | b2 | c1 |
| a2 |    | c2 |
| a2 | b2 | c1 |
| a1 | b2 |    |

The input data is assumed to be categorical and the domain will be inferred from the input dataset:

- `A` possible values are - `a1,a2`
- `B` possible values are - `b1,b2`
- `C` possible values are - `c1,c2`

For a `reporting length=2`, the aggregates in the dataset above could be:

- 1-counts
    - `A:a1`: 3 + NOISE
    - `A:a2`: 2 + NOISE
    - `B:b1`: 1 + NOISE
    - `B:b2`: 3 + NOISE
    - `C:c1`: 3 + NOISE
    - `C:c2`: 1 + NOISE

- 2-counts:
    - `A:a1;C:c1`: 2 + NOISE
    - `A:a2;B:b2`: 1 + NOISE
    - `B:b1;C:c1`: 1 + NOISE
    - `A:a1;B:b1`: 1 + NOISE
    - `A:a1;B:b2`: 2 + NOISE
    - `B:b2;C:c1`: 2 + NOISE
    - `A:a2;C:c2`: 1 + NOISE
    - `B:b2;C:c2`: 0 + NOISE

Also, some spurious combinations might be created and reported to ensure differentially private guarantees - notice that `B:b2;C:c2` does not exist in the sensitive dataset, but it has been _fabricated_ and added to the output.

Following the same lines, some attribute combinations might be suppressed - even though `A:a2;C:c1` exists in the sensitive dataset, it has not been reported as an aggregate.

### 1.2. Synthesis
Data will be then synthesized directly from the aggregates computed with differential privacy to produce synthetic data. Which will ensure the same DP guarantees to the synthetic data.

## 2. Imports and global config

In [22]:
import pandas as pd

from sdssynth import init_logger, set_number_of_threads
from sdssynth import Dataset
from sdssynth import DpAggregateSeededParametersBuilder, AccuracyMode, FabricationMode
from sdssynth import DpAggregatedSeededSynthesizer, Dataset

from utils import gen_dataset, ErrorReport

# The library allows the desired log level to be set if wanted
# ('off' || 'error' || 'warn' || 'info' || 'debug' || 'trace')
# init_logger('trace')

# some algorithms have parallel implementations, so the desired number of threads can be set
# (the default is one thread per CPU core)
# set_number_of_threads(2)

## 3. Generating an example data frame with random data

> `gen_dataset` is just an utility to generate some example data (the code for it is in [`utils.py`](./utils.py))

To exemplify the library, let's create an example data frame that will be used to later on.

In [23]:
number_of_records_to_generate = 6000

sensitive_df = gen_dataset(number_of_records_to_generate)

sensitive_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,2,3,3,1,1,1,0,0,0,0
1,,3,3,0,0,1,1,0,0,1
2,,1,2,1,0,1,0,1,1,0
3,,,2,1,0,1,1,0,1,0
4,2,,4,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
5995,,5,,0,0,1,1,1,1,0
5996,1,5,6,0,1,0,1,0,1,0
5997,,6,9,1,1,0,0,1,0,1
5998,,5,10,1,1,1,0,0,1,0


## 4. Creating the sensitive dataset

### 4.1. Creation from constructor

The library uses an internal representation of the data to optimize the execution time complexity.

To create a dataset you can directly call the construction if the data in the raw format:

In [24]:
sensitive_raw_data = [
    # headers
    ['A', 'B', 'C', 'D'],
    # records
    ['a1', 'b1', 'c1', '0'],
    ['a1', 'b1', 'c2', '0'],
    ['a2', '', 'c2', '1'],
]
sensitive_dataset = Dataset(sensitive_raw_data)

### 4.2. Negative value interpretation

The library distinguishes 'positive' attribute values that indicate the presence of specific sensitive data from 'negative' attribute values that indicate the absence of such data. By default, the integer zero (`0`) and the empty string (`""`) are not taken into account when creating and counting attribute combinations. Any columns where zero values are of interest (and thus sensitive) should be listed as `sensitive_zeros`, so they will be treated the same way as positive values.

> For more parameters see the library documentation.

In [25]:
sensitive_raw_data = [
    # headers
    ['A', 'B', 'C', 'D'],
    # records
    ['a1', 'b1', 'c1', '0'],
    ['a1', 'b1', 'c2', '0'],
    ['a2', '', 'c2', '1'],
]
sensitive_dataset = Dataset(sensitive_raw_data, sensitive_zeros=['D'])

### 4.3. Creating sensitive dataset from a pandas data frame
For convenience a method is provided to build a dataset from a pandas data frame:

In [26]:
sensitive_dataset = Dataset.from_data_frame(sensitive_df)

## 5. Generating the synthetic data

### 5.1. Defining synthesizer parameters

If you just want to create the synthesizer with default parameters:

In [27]:
synth = DpAggregatedSeededSynthesizer()
print (synth.parameters)

{
  "reporting_length": 3,
  "epsilon": 0.1,
  "delta": null,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon": 0.1,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": null
}


Although, this might not produce the optimized output for your dataset.

To tune the synthesizer parameters we provide a builder (`DpAggregateSeededParametersBuilder`), where only the desired parameters can be provided and to have its default value updated:

In [28]:
# this explicitly outlines the default parameters
synth = DpAggregateSeededParametersBuilder() \
        .reporting_length(3) \
        .epsilon(6.0) \
        .delta(1 / (2.0 * len(sensitive_df))) \
        .percentile_percentage(99) \
        .percentile_epsilon_proportion(0.01) \
        .accuracy_mode(AccuracyMode.prioritize_large_counts()) \
        .number_of_records_epsilon(0.1) \
        .fabrication_mode(FabricationMode.uncontrolled()) \
        .empty_value("") \
        .weight_selection_percentile(95) \
        .use_synthetic_counts(False) \
        .aggregate_counts_scale_factor(1.0) \
        .build()
print (synth)


{
  "reporting_length": 3,
  "epsilon": 6.0,
  "delta": 0.00008333333333333333,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon": 0.1,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": 1.0
}


To continue with this example, let's set the parameters we care about for now:

In [29]:
reporting_length = 4

builder = DpAggregateSeededParametersBuilder() \
    .reporting_length(reporting_length) \
    .epsilon(0.2) \
    .accuracy_mode(AccuracyMode.prioritize_large_counts()) \
    .fabrication_mode(FabricationMode.uncontrolled()) \
    .use_synthetic_counts(True)

synth = DpAggregatedSeededSynthesizer(builder.build())

### 5.2 Building the model and synthesizing data

In [30]:
synth.fit(sensitive_dataset)

# here if we do not specify the desired number of samples, the synthesizer will
# use all the available attributes based in the 1-counts to synthesize records
synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

# as an example, let's create a pandas data frame from the raw synthetic data
synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,2,,8,1,,,,,,
5996,2,,8,1,,,,,,
5997,2,,8,1,,,,,,
5998,2,,8,1,,,,,,


## 6. Generating/exporting aggregated data

In [31]:
# generate sensitive aggregates
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')

# export the differentially private aggregates (internal to the synthesizer)
dp_aggregates = synth.get_dp_aggregates(';')

# generate aggregates from the synthetic data
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

# let's take a look at the DP aggregates
dp_aggregates

{'H1:1;H3:3;H6:1;H8:1': 35,
 'H2:5;H3:9;H6:1;H8:1': 2,
 'H1:1;H3:7;H6:1': 168,
 'H2:6;H3:9;H8:1': 56,
 'H1:1;H2:1;H3:1;H4:1': 3,
 'H10:1;H3:6;H8:1': 161,
 'H1:2;H2:5;H4:1': 187,
 'H10:1;H2:1;H3:3;H4:1': 26,
 'H10:1;H3:5;H4:1;H6:1': 76,
 'H2:3;H3:8;H4:1;H7:1': 12,
 'H1:2;H2:6;H3:7;H4:1': 1,
 'H10:1;H1:2;H5:1;H6:1': 258,
 'H10:1;H3:7;H5:1': 138,
 'H2:4;H3:10;H4:1;H9:1': 35,
 'H1:2;H2:2;H3:5;H9:1': 120,
 'H2:2;H3:10;H4:1': 24,
 'H2:2;H3:2;H5:1;H8:1': 8,
 'H3:10;H4:1;H6:1;H7:1': 25,
 'H1:1;H2:2;H7:1;H8:1': 1,
 'H10:1;H1:1;H6:1;H8:1': 251,
 'H1:1;H3:8;H4:1': 30,
 'H1:1;H7:1;H8:1': 593,
 'H1:2;H2:4;H3:5': 24,
 'H1:2;H5:1;H6:1;H9:1': 309,
 'H2:2;H3:10;H4:1;H5:1': 5,
 'H10:1;H2:6;H3:8;H8:1': 38,
 'H1:2;H2:4;H3:9;H7:1': 5,
 'H2:3;H4:1;H6:1;H8:1': 41,
 'H3:6;H4:1;H9:1': 138,
 'H1:1;H2:6;H3:8;H7:1': 30,
 'H10:1;H3:6;H4:1;H9:1': 29,
 'H2:3;H3:2;H4:1;H9:1': 51,
 'H3:7;H6:1;H7:1;H8:1': 89,
 'H1:1;H2:4;H3:1;H5:1': 12,
 'H3:7;H5:1;H8:1': 122,
 'H10:1;H1:1;H2:2;H8:1': 69,
 'H3:10;H4:1;H5:1': 165,
 'H1:

## 7. Evaluate DP aggregates and Synthetic data

This section is an example evaluation of both the DP aggregates and the Synthetic data, as well as the influence of some synthesizer parameters in the DP aggregates and synthetic data.

### 7.1. Evaluating current results

> `ErrorReport` is just an example way to evaluate results (the code for it is [`utils.py`](./utils.py))

- **Count**: mean of the aggregate counts for the given length
- **Error**: mean of the `abs(sensitive_count - dp_aggregated_count)` or `abs(sensitive_count - synthetic_count)`
- **Suppressed %**: percentage of combinations present in the sensitive dataset, but not present in the aggregated/synthetic data
- **Fabricated %**: percentage of combinations that were reported in the aggregated/synthetic data, but do not exist in the sensitive dataset

In [32]:
sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive x DP Aggregates**

In [33]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1385.28 +/- 39.44,0.00 %,0.00 %
1,2,428.04 +/- 52.45,0.48 %,7.14 %
2,3,150.21 +/- 43.82,9.37 %,6.80 %
3,4,56.75 +/- 26.81,39.51 %,3.28 %
4,All,112.19 +/- 34.66,29.17 %,4.76 %


**Sensitive x Synthetic Data**

In [34]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1385.28 +/- 181.92,0.00 %,0.00 %
1,2,428.04 +/- 127.62,0.48 %,7.14 %
2,3,150.21 +/- 71.85,9.37 %,6.59 %
3,4,56.75 +/- 37.41,40.47 %,2.94 %
4,All,112.19 +/- 57.88,29.82 %,4.51 %


### 7.2. Targeting less fabrication

Let's update the current synthesizer parameters to `minimize` fabrication:

In [35]:
synth = DpAggregatedSeededSynthesizer(
    builder \
        .fabrication_mode(FabricationMode.minimize()) \
        .build()
)

synth.fit(sensitive_dataset)

synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,2,,7,,,,,,,
5996,2,,7,,,,,,,
5997,2,,7,,,,,,,
5998,2,,7,,,,,,,


Evaluating again:

In [36]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')
dp_aggregates = synth.get_dp_aggregates(';')
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive x DP Aggregates**

In [37]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1385.28 +/- 72.68,0.00 %,0.00 %
1,2,428.04 +/- 42.04,14.35 %,0.00 %
2,3,150.21 +/- 59.36,71.66 %,0.00 %
3,4,56.75 +/- 50.85,93.34 %,0.00 %
4,All,112.19 +/- 52.72,82.43 %,0.00 %


**Sensitive x Synthetic Data**

In [38]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1385.28 +/- 480.84,0.00 %,0.00 %
1,2,428.04 +/- 278.03,14.35 %,0.00 %
2,3,150.21 +/- 161.96,71.66 %,0.00 %
3,4,56.75 +/- 75.51,93.34 %,0.00 %
4,All,112.19 +/- 186.16,82.43 %,0.00 %


### 7.3. Prioritize small counts

Let's update the current synthesizer parameters to `prioritize_small_counts`:

In [39]:
synth = DpAggregatedSeededSynthesizer(
    builder \
        .fabrication_mode(FabricationMode.uncontrolled()) \
        .accuracy_mode(AccuracyMode.prioritize_small_counts()) \
        .build()
)

synth.fit(sensitive_dataset)

synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,,1,1,1,1
1,1,,,1,1,,1,1,1,1
2,1,,,1,1,,1,1,1,1
3,1,,,1,1,,1,1,1,1
4,1,,,1,1,,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,2,3,9,,,,,,,
5996,2,3,9,,,,,,,
5997,2,6,3,,,,,,,
5998,2,6,3,,,,,,,


Evaluating again:

In [40]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')
dp_aggregates = synth.get_dp_aggregates(';')
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive x DP Aggregates**

In [41]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1385.28 +/- 15.76,0.00 %,0.00 %
1,2,428.04 +/- 33.60,0.00 %,5.86 %
2,3,150.21 +/- 52.13,13.78 %,7.02 %
3,4,56.75 +/- 41.12,57.86 %,4.01 %
4,All,112.19 +/- 44.29,42.69 %,5.34 %


**Sensitive x Synthetic Data**

In [42]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1385.28 +/- 292.80,0.00 %,0.00 %
1,2,428.04 +/- 191.62,0.00 %,5.86 %
2,3,150.21 +/- 104.46,13.78 %,6.24 %
3,4,56.75 +/- 57.39,59.20 %,3.56 %
4,All,112.19 +/- 92.94,43.59 %,4.83 %
